In [1]:
import cv2
import torch
import pandas as pd
from datetime import datetime
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO(r"D:\project\yolo_training\tool_tracking7\weights\best.pt")  # Replace with your trained model

# Open video capture (0 for webcam, or provide video file path)
cap = cv2.VideoCapture(0)

# CSV file setup
csv_filename = "tool_tracking_log.csv"
columns = ["Timestamp", "Tools", "Confidence Scores", "Total Tools", "Highest Confidence Tool", "Highest Confidence Score"]
df = pd.DataFrame(columns=columns)

# Color mapping for different tools
color_map = {}

# Function to generate distinct colors
def get_color(tool_name):
    if tool_name not in color_map:
        color_map[tool_name] = tuple(int(x) for x in tuple((torch.rand(3) * 255).tolist()))
    return color_map[tool_name]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Perform inference
    results = model(frame)[0]
    
    # Apply Non-Maximum Suppression (NMS)
    nms_threshold = 0.4
    boxes, confs, class_ids = [], [], []
    
    for box, conf, cls in zip(results.boxes.xyxy, results.boxes.conf, results.boxes.cls):
        boxes.append(box.tolist())
        confs.append(float(conf))
        class_ids.append(int(cls))
    
    indices = cv2.dnn.NMSBoxes(boxes, confs, 0.5, nms_threshold)
    
    detected_tools = []
    confidence_scores = []
    
    highest_confidence = 0
    highest_conf_tool = ""

    for i in indices:
        x1, y1, x2, y2 = map(int, boxes[i])  # Get bounding box coordinates
        confidence = confs[i]  # Get confidence score
        tool_name = results.names[class_ids[i]]
        
        # Track highest confidence tool
        if confidence > highest_confidence:
            highest_confidence = confidence
            highest_conf_tool = tool_name

        detected_tools.append(tool_name)
        confidence_scores.append(f"{tool_name}: {confidence:.2f}")
        
        # Draw bounding box
        color = (0, 255, 0) if tool_name == highest_conf_tool else get_color(tool_name)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        cv2.putText(frame, f"{tool_name} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Save to CSV
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    total_tools = len(set(detected_tools))
    
    df.loc[len(df)] = [timestamp, ", ".join(set(detected_tools)), ", ".join(confidence_scores), total_tools, highest_conf_tool, highest_confidence]
    df.to_csv(csv_filename, index=False)

    # Display output
    cv2.imshow("YOLOv8 Surgical Tool Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 69.9ms
Speed: 3.2ms preprocess, 69.9ms inference, 34.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 38.1ms
Speed: 2.6ms preprocess, 38.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 38.9ms
Speed: 2.6ms preprocess, 38.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 38.5ms
Speed: 1.8ms preprocess, 38.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 34.1ms
Speed: 1.8ms preprocess, 34.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 35.4ms
Speed: 1.6ms preprocess, 35.4ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 34.6ms
Speed: 2.5ms preprocess, 34.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 39.8ms
Speed: 2.1ms preprocess, 39.8ms 